In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
pip install xlsxwriter

     |████████████████████████████████| 153kB 4.7MB/s 


In [ ]:
import re
import string # inorder to make the category in camel case
import bs4   # in order to use BeautifulSoup
import requests    #in order to extract the read the contents of the url
import urllib.request
from openpyxl import load_workbook,Workbook
import xlsxwriter
import os
#from os import startfile
import openpyxl # 

image_count=0
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent}
file="gdrive/My Drive/Retail/big_basket.xlsx"


# Start by opening the spreadsheet and selecting the main sheet and appending the values in it
def save_into_excel(new_row):
    if os.path.exists(file):
        workbook = load_workbook(filename=file)
    else:
        workbook = Workbook()
        #sheet = workbook.active
        workbook.save(filename=file)
        workbook = load_workbook(filename=file)
    #print(workbook.sheetnames)
    ws = workbook['Sheet']     # Older method was  .get_sheet_by_name('Sheet1')
    row = ws.max_row + 1
    for col, entry in enumerate(new_row, start=1):
        ws.cell(row=row, column=col, value=entry)
    workbook.save(file)
    #startfile(file)



#function to store the images in a folder with category name
def store(category,image_url,name,id,CategoryInfo,Brand,manu,sub_folder,main_folder):
    global image_count
    name=name.replace("/","-")
    for ind in range(len(image_url)):
        request=urllib.request.Request(image_url[ind],None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read()
        MYDIR = category

        CHECK_FOLDER = os.path.isdir("gdrive/My Drive/Retail/{}/{}/{}".format(main_folder,sub_folder,MYDIR))
        # If folder doesn't exist, then create it.
        if not CHECK_FOLDER:
            os.makedirs("gdrive/My Drive/Retail/{}/{}/{}".format(main_folder,sub_folder,MYDIR))
            #print("Folder created: ", MYDIR)
            path="gdrive/My Drive/Retail/{}/{}/{}/{}_{}_{}.jpg".format(main_folder,sub_folder,MYDIR,name,id,ind)
            save_into_excel([CategoryInfo,category,Brand,name+"_"+str(id)+"_"+str(ind),manu])
            output=open(path,'wb')
            output.write(data)
            image_count+=1
            print("images scraped until now",image_count)
            output.close()


        else:
            #print(MYDIR, "folder already exists.")

            path="gdrive/My Drive/Retail/{}/{}/{}/{}_{}_{}.jpg".format(main_folder,sub_folder,MYDIR,name,id,ind)
            #f=".\{}.jpg".format(name)#\path_of_file
            #print("condition",os.path.exists(path))
            if not os.path.exists(path):
                #path="gdrive/My Drive/Retail/{}/{}_{}_{}.jpg".format(MYDIR,name,id,ind)
                save_into_excel([CategoryInfo,category,Brand,name+"_"+str(id)+"_"+str(ind),manu])
                output=open(path,'wb')
                output.write(data)
                image_count+=1
                print("images scraped until now",image_count)
                output.close()

#extract the manufacturer name
def extract_manufacturer(man):
    pat=re.compile("(Pvt. LTD.|Private LTD|Pvt.|,|Pvt. Ltd.)")
    man="".join(man.split('\n')[1:])
    #print(man)
    try:
        start=man.index("Manufact")+8
        start+=man[start:].index(":")
    except:
        start=man.index("Manufactured by:")+16
    try:
        end=list(re.finditer(pat,man[start:]))[0].span()[1]
        s=man[start:start+end]
    except:
        print("error")
        return ""
    return s[1:].strip()

def scrape(url,id): # main function
      try:
        req=requests.get(url) # it grabs the url
        soup = bs4.BeautifulSoup(req.text)
        CategoryInfo = soup.find("div", {"class": "_3moNK"}).text #it finds the div whose class is '_3moNK' and collects all the text
        Manufacturer=soup.find_all("div", {"style":"font-family: 'ProximaNova-Regular';font-size:13px;line-height: 18px;color:8f8f8f;"})#.text
        Brand=soup.find("a",{"class":"_2zLWN _3bj9B rippleEffect"}).text
        #print("Brand is:",Brand)
        #type(Manufacturer[-1])
          
        try:
          manu=""
          man=str(Manufacturer[-1])
          manu=extract_manufacturer(man)
          #print("Manufacturer is:",manu)
        except:

          print("unable to extract manufacturer")
          
        Cat_split=CategoryInfo.split('>') # splits all the text based on the delimeter '>'
        #print(Cat_split)
        if len(Cat_split)>=2:
                main_folder=string.capwords(Cat_split[-4])
                sub_folder=string.capwords(Cat_split[-3])
                category=Cat_split[-2] #it selects the second last element which contains the category of the product
                name=string.capwords(Cat_split[-1]) #select the last element which contains name of the product
                imagee=[]
                image_url=soup.find_all("div", {"class": "_2FbOx"})#.img['src'] # selects the div which contains class ="_2FbOx" and grabs the imag from 'src'
                for im in image_url[:2]:
                    imagee.append(im.img['src'])
                    #print("image",im)
                #PriceInfo = soup.find("div", {"id": "price"}).text   #selects the div which contains id ="price" and grabs the text info'
                #price=PriceInfo[PriceInfo.find('Price:')+9:PriceInfo.find('You')] # selecting only price from the selected text
                print("category is",category)
                print("name of the product",name)
                #print("url of the product image",image_url)
                #print("price of the product",price)
                store(string.capwords(category),imagee,name,id[4:-1],"".join(CategoryInfo),Brand,manu,sub_folder,main_folder)

                #save_into_excel(["".join(CategoryInfo),string.capwords(category),Brand,name,manu])
        else:
              print("Nothing is there in the",url,end=" ")
              print(Cat_split)
      except Exception as e:
        print(e)
        print("Invalid ",url)

if __name__=="__main__":
  slug,i=input("enter the category and sarting page").split()
  i=int(i)
  count=0
  while True:
    try:
      if i==1:
        url="https://www.bigbasket.com/custompage/sysgenpd/?type=pc&slug={}".format(slug)
      else:
        url="https://www.bigbasket.com/product/get-products/?slug={}&page={}&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc".format(slug,i) 
      res = requests.get(url)
    except Exception as e:
      print("invalid category name",e) 
    soup = bs4.BeautifulSoup(res.text)
    soup=str(soup)

    pat=re.compile("\/pd\/[0-9]*\/")
    p=list(re.findall(pat,soup))
    if len(p)==0:
      print("Completely extracted this category, now move to next category")
      print("Total number of images scraped are",image_count)
      break

    count+=len(p)
    if count>1000:
      print("Total number of images scraped are",image_count)
      print("In next iteration please start with these new input",slug,i)
      break
    print("PAGE NO ",i)
    for product in p:
      scrape("https://www.bigbasket.com"+product,product)
      
    i+=1
    
    

enter the category and sarting pagebeverages 24
PAGE NO  24
category is JUICES
name of the product Kapiva Ayurveda Triphala Juice
category is JUICES
name of the product Slice Thickest Mango Drink
category is JUICES
name of the product Tropicana Slice Mango Juice
category is JUICES
name of the product Tropicana Slice Mango Juice
category is JUICES
name of the product Kapiva Ayurveda Noni Juice
category is JUICES
name of the product Brewhouse Tea Brewing Co. Organic Forest Berry Ice Tea
category is LEAF & DUST TEA
name of the product Tata Tea Chakra Gold Activ+ Tea
category is LEAF & DUST TEA
name of the product Tata Tea Chakra Gold Activ+ Tea
category is LEAF & DUST TEA
name of the product Tata Tea Chakra Gold Activ+ Tea
unable to extract manufacturer
category is GROUND COFFEE
name of the product Coffee Day Coffee - 7 Day
category is GREEN TEA
name of the product Lipton Green Tea - Honey, Lemon
category is JUICES
name of the product B Natural Juice - Pink Guava
category is TEA BAGS
name

In [ ]:
snacks-branded-foods 26

In [ ]:
#Don't run this code
'''
import re
import requests
import bs4
if __name__=="__main__":
  count=0
  slug,i=input("enter the category and starting page with space in between").split()
  i=int(i)
  while True:
    try:
      if i==1:
        url="https://www.bigbasket.com/custompage/sysgenpd/?type=pc&slug={}".format(slug)
      else:
        url="https://www.bigbasket.com/product/get-products/?slug={}&page={}&tab_type=[%22all%22]&sorted_on=popularity&listtype=pc".format(slug,i)
      res = requests.get(url)
    except Exception as e:
      print("invalid category name",e) 
    soup = bs4.BeautifulSoup(res.text)
    soup=str(soup)

    pat=re.compile("\/pd\/[0-9]*\/")
    p=list(re.findall(pat,soup))
    count+=len(p)
    if len(p)==0:
      print("images scraped so far",count)
      print("start from another category")
      break
    if count>1000:
      print("images scraped so far",count)
      print("start from next ",slug,i)
      break  
    #for product in p:
     # scrape("https://www.bigbasket.com"+product,product)
    
    i+=1 
 '''   